In [1]:
import os, sys
import pathlib
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#!pip install -U --pre tensorflow=="2.*"
!pip install tf_slim
!pip install pycocotools
!pip install keyboard
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

     |████████████████████████████████| 358kB 3.0MB/s 
     |████████████████████████████████| 61kB 2.0MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.17-1~18.04).
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 21 not upgraded.
Need to get 1,791 kB of archives.
After this operation, 7,807 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubu

In [ ]:
# CLONE DE GIT
# try:
#   if "drive/My\ Drive/models" in pathlib.Path.cwd().parts:
#     while "drive/My\ Drive/models" in pathlib.Path.cwd().parts:
#       os.chdir('/content/drive/My Drive')
#   elif not pathlib.Path('drive/My\ Drive/models').exists():
#     os.chdir('/content/drive/My Drive')
#     !git clone --depth 1 https://github.com/tensorflow/models
# except:
#   None

In [3]:
# compile proto files
# make sure you are in BASE_DIR/research/ (see cell below)
%cd /content/drive/My Drive/models/research/
# !protoc object_detection/protos/*.proto --python_out=.
# !cp object_detection/packages/tf2/setup.py .
# !python -m pip install .

/content/drive/My Drive/models/research


In [4]:
print(os.getcwd())
os.chdir("/content/drive/My Drive/models/research/object_detection")
print(os.getcwd())
try:
  DATA_DIR = os.path.join(os.getcwd(), 'data')
  MODELS_DIR = os.path.join(DATA_DIR, 'models')
  for dir in [DATA_DIR, MODELS_DIR]:
    if not os.path.exists(dir):
      os.mkdir(dir)
except:
  print('error')

/content/drive/My Drive/models/research
/content/drive/My Drive/models/research/object_detection


In [5]:
import os
import tarfile
import urllib.request

# os.chdir('/research')

# Download and extract model
MODEL_DATE = '20200711'
#'efficientdet_d2_coco17_tpu-32' #my primary model
#'centernet_resnet101_v1_fpn_512x512_coco17_tpu-8' #secondary model (worst)
#'efficientdet_d3_coco17_tpu-32' #third model (should be finest)
MODEL_NAME = 'efficientdet_d2_coco17_tpu-32'
MODEL_TAR_FILENAME = MODEL_NAME + '.tar.gz'
MODELS_DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/tf2/'
MODEL_DOWNLOAD_LINK = MODELS_DOWNLOAD_BASE + MODEL_DATE + '/' + MODEL_TAR_FILENAME
PATH_TO_MODEL_TAR = os.path.join(MODELS_DIR, MODEL_TAR_FILENAME)
PATH_TO_CKPT = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'checkpoint/'))
PATH_TO_CFG = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'pipeline.config'))
if not os.path.exists(PATH_TO_CKPT):
  print('Downloading model. This may take a while... ', end='')
  urllib.request.urlretrieve(MODEL_DOWNLOAD_LINK, PATH_TO_MODEL_TAR)
  tar_file = tarfile.open(PATH_TO_MODEL_TAR)
  tar_file.extractall(MODELS_DIR)
  tar_file.close()
  os.remove(PATH_TO_MODEL_TAR)
  print('Done')

# Download labels file
LABEL_FILENAME = 'mscoco_label_map.pbtxt'
LABELS_DOWNLOAD_BASE = \
    'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/'
PATH_TO_LABELS = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, LABEL_FILENAME))
if not os.path.exists(PATH_TO_LABELS):
  print('Downloading label file... ', end='')
  urllib.request.urlretrieve(LABELS_DOWNLOAD_BASE + LABEL_FILENAME, PATH_TO_LABELS)
  print('Done')# %%bash

In [6]:
print(os.getcwd())

/content/drive/My Drive/models/research/object_detection


In [7]:
print(PATH_TO_CFG)

/content/drive/My Drive/models/research/object_detection/data/models/efficientdet_d2_coco17_tpu-32/pipeline.config


In [8]:
os.chdir('/content/drive/My Drive/models/research')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
os.chdir('/content/drive/My Drive/models')
from object_detection.builders import model_builder

tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-0')).expect_partial()

@tf.function
def detect_fn(image):
    """Detect objects in image."""

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

In [9]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

In [10]:
print(category_index)

{1: {'id': 1, 'name': 'person'}, 2: {'id': 2, 'name': 'bicycle'}, 3: {'id': 3, 'name': 'car'}, 4: {'id': 4, 'name': 'motorcycle'}, 5: {'id': 5, 'name': 'airplane'}, 6: {'id': 6, 'name': 'bus'}, 7: {'id': 7, 'name': 'train'}, 8: {'id': 8, 'name': 'truck'}, 9: {'id': 9, 'name': 'boat'}, 10: {'id': 10, 'name': 'traffic light'}, 11: {'id': 11, 'name': 'fire hydrant'}, 13: {'id': 13, 'name': 'stop sign'}, 14: {'id': 14, 'name': 'parking meter'}, 15: {'id': 15, 'name': 'bench'}, 16: {'id': 16, 'name': 'bird'}, 17: {'id': 17, 'name': 'cat'}, 18: {'id': 18, 'name': 'dog'}, 19: {'id': 19, 'name': 'horse'}, 20: {'id': 20, 'name': 'sheep'}, 21: {'id': 21, 'name': 'cow'}, 22: {'id': 22, 'name': 'elephant'}, 23: {'id': 23, 'name': 'bear'}, 24: {'id': 24, 'name': 'zebra'}, 25: {'id': 25, 'name': 'giraffe'}, 27: {'id': 27, 'name': 'backpack'}, 28: {'id': 28, 'name': 'umbrella'}, 31: {'id': 31, 'name': 'handbag'}, 32: {'id': 32, 'name': 'tie'}, 33: {'id': 33, 'name': 'suitcase'}, 34: {'id': 34, 'name'

In [11]:
import cv2
import numpy as np

In [12]:
# A Python3 program to find if 2 given line segments intersect or not 
  
class Point: 
    def __init__(self, x, y): 
        self.x = x 
        self.y = y 
  
# Given three colinear points p, q, r, the function checks if  
# point q lies on line segment 'pr'  
def onSegment(p, q, r): 
    if ( (q.x <= max(p.x, r.x)) and (q.x >= min(p.x, r.x)) and 
           (q.y <= max(p.y, r.y)) and (q.y >= min(p.y, r.y))): 
        return True
    return False
  
def orientation(p, q, r): 
    # to find the orientation of an ordered triplet (p,q,r) 
    # function returns the following values: 
    # 0 : Colinear points 
    # 1 : Clockwise points 
    # 2 : Counterclockwise 
      
    # See https://www.geeksforgeeks.org/orientation-3-ordered-points/amp/  
    # for details of below formula.  
      
    val = (float(q.y - p.y) * (r.x - q.x)) - (float(q.x - p.x) * (r.y - q.y)) 
    if (val > 0): 
          
        # Clockwise orientation 
        return 1
    elif (val < 0): 
          
        # Counterclockwise orientation 
        return 2
    else: 
          
        # Colinear orientation 
        return 0
  
# The main function that returns true if  
# the line segment 'p1q1' and 'p2q2' intersect. 
def doIntersect(p1,q1,p2,q2): 
      
    # Find the 4 orientations required for  
    # the general and special cases 
    o1 = orientation(p1, q1, p2) 
    o2 = orientation(p1, q1, q2) 
    o3 = orientation(p2, q2, p1) 
    o4 = orientation(p2, q2, q1) 
  
    # General case 
    if ((o1 != o2) and (o3 != o4)): 
        return True
  
    # Special Cases 
  
    # p1 , q1 and p2 are colinear and p2 lies on segment p1q1 
    if ((o1 == 0) and onSegment(p1, p2, q1)): 
        return True
  
    # p1 , q1 and q2 are colinear and q2 lies on segment p1q1 
    if ((o2 == 0) and onSegment(p1, q2, q1)): 
        return True
  
    # p2 , q2 and p1 are colinear and p1 lies on segment p2q2 
    if ((o3 == 0) and onSegment(p2, p1, q2)): 
        return True
  
    # p2 , q2 and q1 are colinear and q1 lies on segment p2q2 
    if ((o4 == 0) and onSegment(p2, q1, q2)): 
        return True
  
    # If none of the cases 
    return False

In [13]:
def draw_centroid(frame, centroid):
  red = (0, 0, 255)
  value = tuple(centroid)
  cv2.circle(frame, value, 1, red, 10)

def draw_text(frame, centroid, id):
  centroid = tuple(centroid)
  green = (0, 255, 0)
  font = cv2.FONT_HERSHEY_PLAIN
  cv2.putText(frame, "Person #" + str(id), centroid, font, 2, green, 2, cv2.LINE_AA)

def draw_counter_text(frame, count):
  position = (100,100)
  green = (0, 0, 255)
  font = cv2.FONT_HERSHEY_PLAIN
  cv2.putText(frame, "Counter :" + str(count), position, font, 3, green, 3, cv2.LINE_AA)

def draw_in_out_text(frame, count_in, count_out):
  position_in = (100,90)
  position_out = (100,130)
  green = (0, 0, 255)
  font = cv2.FONT_HERSHEY_PLAIN
  text_in = "In : " + str(count_in)
  text_out = "Out : " + str(count_out)
  cv2.putText(frame, text_in, position_in, font, 3, green, 3, cv2.LINE_AA)
  cv2.putText(frame, text_out, position_out, font, 3, green, 3, cv2.LINE_AA)

In [20]:
# The following has been modified to fit TF2's vis util box normalized bounding box
# import the necessary packages
from scipy.spatial import distance as dist
from collections import OrderedDict
import numpy as np

class CentroidTracker():
	def __init__(self, frame_shape, maxDisappeared=50, dualCounter=True):
		# initialize the next unique object ID along with two ordered
		# dictionaries used to keep track of mapping a given object
		# ID to its centroid and number of consecutive frames it has
		# been marked as "disappeared", respectively
		self.nextObjectID = 0
		self.objects = OrderedDict()
		self.disappeared = OrderedDict()
		# Height, width and color channel of the frame
		self.height, self.width, self.channel = frame_shape
		self.count_in = 0  # no. of visitors entering the building
		self.count_out = 0 # no. of visitors exiting the building

		# store the number of maximum consecutive frames a given
		# object is allowed to be marked as "disappeared" until we
		# need to deregister the object from tracking
		self.maxDisappeared = maxDisappeared

		# Display people enterring and exitting separately if set to true
		self.dualCounter = dualCounter

	def register(self, centroid):
		# when registering an object we use the next available object
		# ID to store the centroid
		self.objects[self.nextObjectID] = centroid
		self.disappeared[self.nextObjectID] = 0
		self.nextObjectID += 1

	def deregister(self, objectID):
		# to deregister an object ID we delete the object ID from
		# both of our respective dictionaries
		del self.objects[objectID]
		del self.disappeared[objectID]

	def update(self, frame, rects, count=False, ROI_start_point=None, ROI_end_point=None):

		# If counter displays people enterring and exitting the building 
		# separately
		if self.dualCounter:
			draw_in_out_text(frame, self.count_in, self.count_out) # draw in/out counter
		# else if the counter only displays the number of people in the building
		else:
			counter = self.count_in - self.count_out
			draw_counter_text(frame,counter)  # draw a counter

		# check to see if the list of input bounding box rectangles
		# is empty
		if len(rects) == 0:
			# loop over any existing tracked objects and mark them
			# as disappeared
			for objectID in list(self.disappeared.keys()):
				self.disappeared[objectID] += 1

				# if we have reached a maximum number of consecutive
				# frames where a given object has been marked as
				# missing, deregister it
				if self.disappeared[objectID] > self.maxDisappeared:
					self.deregister(objectID)

			# return early as there are no centroids or tracking info
			# to update
			return self.objects

		# initialize an array of input centroids for the current frame
		inputCentroids = np.zeros((len(rects), 2), dtype="int")

		# loop over the bounding box rectangles
		for (i, (endY, endX, startY, startX)) in enumerate(rects):
			# denormalize the bounding box coordinate
			d_minX = startX * self.height
			d_maxX = endX * self.height
			d_minY = startY * self.width
			d_maxY = endY * self.width
			# use the bounding box coordinates to derive the centroid
			cX = int((d_minX + d_maxX) / 2.0)
			cY = int((d_minY + d_maxY) / 2.0)
			inputCentroids[i] = (cX, cY)

		# if we are currently not tracking any objects take the input
		# centroids and register each of them
		if len(self.objects) == 0:
			for i in range(0, len(inputCentroids)):
				self.register(inputCentroids[i])
				draw_centroid(frame, inputCentroids[i])  # draw centroid if new person is detected AND Tracker is not tracking
				draw_text(frame, inputCentroids[i], self.nextObjectID-1)

		# otherwise, are are currently tracking objects so we need to
		# try to match the input centroids to existing object
		# centroids
		else:
			# grab the set of object IDs and corresponding centroids
			objectIDs = list(self.objects.keys())
			objectCentroids = list(self.objects.values())

			# compute the distance between each pair of object
			# centroids and input centroids, respectively -- our
			# goal will be to match an input centroid to an existing
			# object centroid
			D = dist.cdist(np.array(objectCentroids), inputCentroids)

			# in order to perform this matching we must (1) find the
			# smallest value in each row and then (2) sort the row
			# indexes based on their minimum values so that the row
			# with the smallest value as at the *front* of the index
			# list
			rows = D.min(axis=1).argsort()

			# next, we perform a similar process on the columns by
			# finding the smallest value in each column and then
			# sorting using the previously computed row index list
			cols = D.argmin(axis=1)[rows]

			# in order to determine if we need to update, register,
			# or deregister an object we need to keep track of which
			# of the rows and column indexes we have already examined
			usedRows = set()
			usedCols = set()

			# loop over the combination of the (row, column) index
			# tuples
			for (row, col) in zip(rows, cols):
				# if we have already examined either the row or
				# column value before, ignore it
				# val
				if row in usedRows or col in usedCols:
					continue

				# otherwise, grab the object ID for the current row,
				# set its new centroid, and reset the disappeared
				# counter
				objectID = objectIDs[row]

				# Check to see first if that object has crossed the ROI
				if count:  # counting only starts if it's enabled
					if (self.objects[objectID]).all():
						beforeMove = Point(int(self.objects[objectID][0]), int(self.objects[objectID][1]))
						afterMove = Point(int(inputCentroids[col][0]), int(inputCentroids[col][1]))
					
						if doIntersect(ROI_start_point, ROI_end_point, beforeMove, afterMove):
							# If the current centroid is below the ROI, counter goes up
							# meaning an object has entered the building
							if afterMove.y > ROI_start_point.y:  # if y is higher means it's below in the actual frame
								self.count_in += 1
							else:
								self.count_out -= 1  # if it's the opposite means an object has exited the building

				self.objects[objectID] = inputCentroids[col]
				self.disappeared[objectID] = 0
				draw_centroid(frame, inputCentroids[col])  # draw centroid when a person moved
				draw_text(frame, inputCentroids[col], objectID)

				# indicate that we have examined each of the row and
				# column indexes, respectively
				usedRows.add(row)
				usedCols.add(col)

			# compute both the row and column index we have NOT yet
			# examined
			unusedRows = set(range(0, D.shape[0])).difference(usedRows)
			unusedCols = set(range(0, D.shape[1])).difference(usedCols)

			# in the event that the number of object centroids is
			# equal or greater than the number of input centroids
			# we need to check and see if some of these objects have
			# potentially disappeared
			if D.shape[0] >= D.shape[1]:
				# loop over the unused row indexes
				for row in unusedRows:
					# grab the object ID for the corresponding row
					# index and increment the disappeared counter
					objectID = objectIDs[row]
					self.disappeared[objectID] += 1

					# check to see if the number of consecutive
					# frames the object has been marked "disappeared"
					# for warrants deregistering the object
					if self.disappeared[objectID] > self.maxDisappeared:
						self.deregister(objectID)

			# otherwise, if the number of input centroids is greater
			# than the number of existing object centroids we need to
			# register each new input centroid as a trackable object
			else:
				for col in unusedCols:
					self.register(inputCentroids[col])
					draw_centroid(frame, inputCentroids[col])  # draw a centroid when a new person is detected
					draw_text(frame, inputCentroids[col], self.nextObjectID-1)

		# return the set of trackable objects
		return self.objects

In [15]:
def confidence_pruning(boxes, scores, min_threshold=0.3):
  indices = np.squeeze(np.argwhere(scores >= min_threshold))
  boxes_pruned = boxes[[indices]]
  if boxes_pruned.ndim < 2:
    boxes_pruned = np.expand_dims(boxes_pruned, axis=0)
  return boxes_pruned

In [16]:
import time
from datetime import date

In [21]:
today = str(date.today())

thresh = 0.35 #threshold for filtering person

### INPUT & OUTPUT VIDEO FILES ###
#25 AGST 20 1145 1200.avi #my main vid
#1 sept 2020 1100 1130.avi #second vid
VIDEO_NAME = '1 sept _ kopi oey -12frame.mp4'
VIDEODIR = '/content/drive/My Drive/objDet/vid/' + VIDEO_NAME
OUTDIR = '/content/drive/My Drive/objDet/outvid/' + 'TEST' + today + 'short_thresh:'+str(thresh)  + MODEL_NAME + '__' + VIDEO_NAME

### SETTING UP CV2 & VIDEO ###
cap = cv2.VideoCapture(VIDEODIR) #Getting video for cv2
#default centerDet thresh = 0.55
#centerDet d2 = 0.45 1st run
#centerDet d2 = 0.4  2nd run
#default centerNet thresh = 0.3 ~ 0.35

### VIDEO TECHNICAL ###
ret, image_np = cap.read()
fshape = image_np.shape
fheight = fshape[0]
fwidth = fshape[1]
fps = int(cap.get(cv2.CAP_PROP_FPS))
print("fps :",fps)
res = (fwidth , fheight)
print("res :",res)
#.mp4 format : *'MP4V' ; .avi : *'XVID'
out = cv2.VideoWriter(OUTDIR,cv2.VideoWriter_fourcc(*'MP4V'), fps, res)

### GETTING CENTROID IF ANY DETECTION ###
Tracker = CentroidTracker([fwidth,fheight,3], maxDisappeared=30)

### LINE INTERSECT FOR COUNTING
start_point = (0, int(fheight*0.65))
end_point = (int(fwidth), int(fheight*0.65))
color = (255,0,0)
thickness = 5
ROI_start_point = Point(start_point[0], start_point[1])
ROI_end_point = Point(end_point[0], end_point[1])

### SETTING UP WHEN to START & STOP, SHOWING DURATION ###   <---------------------------------###########################
startDetection = 3 * 60 + 9 #when to start detect
length = 1 * 2 #in second(s)  #Length how long to detect

l_startDetection = startDetection * fps #how many frames to skip detect
cap.set(cv2.CAP_PROP_POS_FRAMES, l_startDetection) #skip frame
l_fps = length * fps #how many frames to detect
i = 0 #initiate first frame

### IT'S ALL BEGIN ###
if not (startDetection == 0):
  print('{} s are skipped!'.format(startDetection))
print('Ready to process!')

label_id_offset = 1

startTime = time.time() #how long it takes to process desired length video

while (i < l_fps): #start to detect
  # Read frame from video
  ret, image_np = cap.read()

  if not ret:
    break

  #Draw a line for counting
  cv2.line(image_np, start_point, end_point, color, thickness)

  input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
  detections, predictions_dict, shapes = detect_fn(input_tensor)

  indices = np.squeeze(np.argwhere((detections['detection_classes'][0].numpy() + label_id_offset).astype(int) == 1))
  boxes = detections['detection_boxes'][0].numpy()
  boxes = boxes[[indices]]
  classes = (detections['detection_classes'][0] + label_id_offset).numpy()
  classes = classes[[indices]].astype(int)
  scores = detections['detection_scores'][0].numpy()
  scores = scores[[indices]]
  boxes = confidence_pruning(boxes, scores, thresh)

  People = Tracker.update(image_np, boxes, count=True, ROI_start_point=ROI_start_point, ROI_end_point=ROI_end_point)

  print("\r progress: {} s / {} s".format(str(int(((i+1)/fps))),length), end =" ")

  # Display output
  #nFrame = cv2.resize(image_np_with_detections, (800, 600))
  #cv2.imshow('object detection', nFrame)
  
  # write the frame
  out.write(image_np)

  i = i+1

cap.release()
out.release()
doneTime = time.time()-startTime
print("")
print("DONE!")
print("Saved as :",OUTDIR)
print("Finish in : {} (s)".format(float(doneTime)))

fps : 12
res : (1920, 1080)
189 s are skipped!
Ready to process!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:75: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:77: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:79: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a differe

 progress: 1 s / 2 s 0
 progress: 1 s / 2 s 0
 progress: 1 s / 2 s 0
 progress: 1 s / 2 s 0
 progress: 1 s / 2 s 0
 progress: 1 s / 2 s 0
 progress: 1 s / 2 s 0
 progress: 1 s / 2 s 0
 progress: 1 s / 2 s 0
 progress: 1 s / 2 s 0
 progress: 1 s / 2 s 0
 progress: 1 s / 2 s 0
 progress: 2 s / 2 s 
DONE!
Saved as : /content/drive/My Drive/objDet/outvid/TEST2020-10-18short_thresh:0.35efficientdet_d2_coco17_tpu-32__1 sept _ kopi oey -12frame.mp4
Finish in : 9.952859878540039 (s)
